In [ ]:
import pandas as pd
from utils.lngselection import abbreviation
from wikiwho_wrapper import WikiWho
from external.wikipedia import WikipediaDV, WikipediaAPI
from metrics.conflict import ConflictManager
import numpy as np
import pyLDAvis
import numpy as np

In [ ]:
%%capture
## Some Extensions ##
%load_ext autoreload
%autoreload 2
%store -r the_page

if 'the_page' not in locals():
    import pickle
    print("Loading default data...")
    the_page = pickle.load(open("data/the_page.p",'rb'))

lng = abbreviation('English')

In [ ]:
wikiwho = WikiWho(lng=lng)
all_content = wikiwho.dv.all_content(the_page['page_id'])
revisions = wikiwho.dv.rev_ids_of_article(the_page['page_id'])

In [ ]:
con_manager = ConflictManager(all_content.copy(), 
                                           revisions.copy(), 
                                           lng=lng, 
                                           include_stopwords=False)

In [ ]:
con_manager.calculate()
token = con_manager.all_actions.copy()
tokens_processed = token[['rev_id', 'rev_time', 'editor', 'token']].groupby("rev_id")['token'].apply(lambda group_series: group_series.tolist()).reset_index()
tokens_processed['token']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def dummy(doc):
    return doc

vec = CountVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
    )  

tokens_processed = tokens_processed[np.array(list(map(len,tokens_processed.token.values)))>1]
#tokens_processed.drop(443, inplace=True)
#tokens_processed.reset_index(inplace=True)
X = vec.fit_transform(tokens_processed['token'].tolist()).toarray()

In [ ]:
from biterm.utility import vec_to_biterms, topic_summuary

vocab = np.array(vec.get_feature_names())
biterms = vec_to_biterms(X)

In [ ]:
from biterm.btm import oBTM

btm = oBTM(num_topics=20, V=vocab) #default alpha=1., beta=0.01, l=0.5
topics = btm.fit_transform(biterms, iterations=100)

In [ ]:
new_topics = [topic for topic in topics if not np.isnan(topic).any()]

In [ ]:
vis = pyLDAvis.prepare(btm.phi_wz.T, new_topics, np.count_nonzero(X, axis=1), vocab, np.sum(X, axis=0))

#pyLDAvis.save_html(vis, 'online_btm.html')  # path to output

print("\n\n Topic coherence ..")
topic_summuary(btm.phi_wz.T, X, vocab, 10)


In [ ]:
pyLDAvis.display(vis)

In [ ]:
import math
for i in range(len(topics)):
    if math.isnan(topics[i].sum()):
        print(i)

In [ ]:
tokens_processed['token'].tolist()[443]